# Figure2(Stackedbar) for bertopic

In [ ]:
import sys, os
p = os.path.abspath('../..')
sys.path.insert(1, p)

# pp = os.path.abspath('../../../visualization')
# sys.path.insert(1, pp)

In [ ]:
from gc import collect

import pickle
import lib.stats.stats as st
from utils.statistics import *
from utils.settings import set_matplotlib
from utils.distribution_collector import (collect_topic_distributions,
                                get_top_and_bottom_topics,
                                extract_specific_topics)
from constants import CONSTANTS
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from statsmodels.nonparametric.smoothers_lowess import lowess
import matplotlib as mpl

import psycopg2
# from utils.statistics import *
import config.config as conf
import datetime
import re
# 포뮬러 구성
import statsmodels.formula.api as smf
import numpy as np
import pandas as pd



In [ ]:
from matplotlib import font_manager
import matplotlib as mpl
from matplotlib import font_manager as fm

# 설치된 폰트 이름과 경로 확인
for font in font_manager.fontManager.ttflist:
    if 'Helvetica' in font.name: # 'Nanum' 대신 원하는 글꼴의 일부를 입력하여 검색할 수 있습니다.
        print(font.name, font.fname)

# 직접 경로로 Helvetica 폰트 불러오기
font_path = "/System/Library/Fonts/Helvetica.ttc"
font_prop = fm.FontProperties(fname=font_path)
font_name = font_prop.get_name()
print(f"Registered font name: {font_name}")

mpl.rcParams['font.family'] = font_name


In [ ]:
data_dir = f'../../../so_pjt_src/{CONSTANTS.bert_monthly_data_dir_3[3:]}'
output_dir = './fig/'
date_range = 'Weekly'
model = 'BERT'

In [ ]:
data_dir

In [ ]:
lst = os.listdir(data_dir)
df = pd.DataFrame()
# 디렉토리 내의 파일을 for 문으로 반복
for i in lst:
	js = load_json(f'{data_dir}/{i}')
	tmp = pd.DataFrame(js)
	tmp['json'] = i
	df = pd.concat([df, tmp], axis = 0)

In [ ]:
df = df[['id', 'creationdate', 'Topic', 'json']].copy()

In [ ]:
df['creationdate'] = pd.to_datetime(df['creationdate'], format="mixed")
df['cdate'] = pd.to_datetime(df['creationdate'], format="%Y-%m-%d").dt.date
df['cdate'] = pd.to_datetime(df['cdate'], format="%Y-%m-%d")


In [ ]:
topic_list = list(df[(df['cdate']>=CONSTANTS.start_date_snap1)&\
                 (df['cdate']<CONSTANTS.end_date_snap1)]\
                .groupby('Topic').count()['id'].reset_index().sort_values(by = 'id', ascending=False)['Topic'])


top10list = topic_list[:10]
bot10list = topic_list[-10:]



In [ ]:
snap1_df = df[df['cdate'] <  CONSTANTS.end_date_snap1]
snap2_df = df[df['cdate'] >= CONSTANTS.start_date_snap2]
tot_df   = df[(df['cdate']>= CONSTANTS.start_date_snap1) & (df['cdate']< CONSTANTS.end_date_snap2)]

In [ ]:
snap1_df['rel_week'] = np.floor((snap1_df['cdate']-\
                       CONSTANTS.std_date_snap1).dt.days/7) 
snap2_df['rel_week'] = np.floor((snap2_df['cdate']-\
                       CONSTANTS.std_date_snap2).dt.days/7) 
tot_df['rel_week'] = np.floor((tot_df['cdate']-\
                       CONSTANTS.std_date_snap1).dt.days/7) 

In [ ]:
def proportion_calc(df):
    df_tot = df.groupby('rel_week').count()['id'].reset_index(name = 'tot_cnt')
    df_topic = df.groupby(['rel_week', 'Topic']).count()['id'].reset_index(name = 'cnt')
    df_topic_r = pd.merge(df_topic, df_tot, on = 'rel_week')
    df_topic_r['pct'] = df_topic_r['cnt']/df_topic_r['tot_cnt']

    df_topic_r_t = df_topic_r[df_topic_r['Topic'].isin(top10list)]
    df_topic_r_b = df_topic_r[df_topic_r['Topic'].isin(bot10list)]

    proportion_dict = {'Top 20% Topics' : df_topic_r_t, 'Bottom 20% Topics' : df_topic_r_b}
    list_10 = {'Top 20% Topics' : top10list, 'Bottom 20% Topics' : bot10list}
    return proportion_dict, list_10


In [ ]:
snap1_dict, list_= proportion_calc(snap1_df)
snap2_dict, list_= proportion_calc(snap2_df)
tot_dict,   list_= proportion_calc(tot_df)

In [ ]:
import seaborn as sns
color_list = [[ 
   "#4575b4",  # deep blue
    "#91bfdb",  # light blue
    "#e0f3f8",  # pale blue
    "#a6d96a",  # light green
    "#1a9850",  # green
    "#d9ef8b",  # lime yellow
    "#fee08b",  # beige
    "#fdae61",  # soft orange
    "#f46d43",  # coral orange
    "#d73027"   # muted red
    ],
   [
    "#8c510a",  # dark brown
    "#bf812d",  # brown-gold
    "#dfc27d",  # sand yellow
    "#f6e8c3",  # beige
    "#c7eae5",  # light aqua
    "#80cdc1",  # teal
    "#35978f",  # muted teal
    "#01665e",  # deep green
    "#003c30",  # near-black green
    "#f5f5f5"   # pale gray (neutral base)
]]


In [ ]:

### create a figure with multiple subplots
sharey = True ## 또는 sharey=False
sharex = True ## 또는 sharex=False
# g_num  = len(proportion_dict.items())

fig, axs = plt.subplots(1, 2, figsize = (18, 6), constrained_layout=True)
colors = plt.get_cmap('tab20').colors 


for x, (title, proportion) in enumerate(tot_dict.items()):
    rel_week = sorted(proportion['rel_week'].unique())  # 주차 순서 보장
    topic_list = list(proportion['Topic'].unique())
    order_list = list_[title]
    colors = color_list[x]

    # 전체 주차 길이에 맞춰 bottom 초기화
    bottom = np.zeros(len(rel_week))
    

    for idx, topic in enumerate(order_list):
        # topic별 데이터
        t_p = proportion[proportion['Topic'] == topic]

        # topic별 주차별 비율을 rel_week 길이에 맞춰 채우기
        count_full = np.zeros(len(rel_week))
        for i, rw in enumerate(t_p['rel_week']):
            if rw in rel_week:
                rw_idx = rel_week.index(rw)
                count_full[rw_idx] = t_p.loc[t_p['rel_week'] == rw, 'pct'].values[0]

        # bar plot
        axs[x].bar(rel_week, count_full, bottom=bottom, label=topic, color=colors[idx], width=1.0, align='center')

        bottom += count_full  # 누적

    # 보조선 및 제목/레이블 등
    axs[x].axvline(x=0, color='tab:red', linestyle='-.', linewidth=1)
    axs[x].set_title(f'{title}', fontsize=25)
    axs[x].tick_params(axis='x', labelsize=16)
    axs[x].tick_params(axis='y', labelsize=16)

axs[0].set_ylabel("Accumulated topic share", fontsize = 22)


fig.supxlabel("Week relative to ChatGPT release", fontsize=22) 

plt.savefig(f"{output_dir}C_Result_Fig2_2.png", dpi=300, bbox_inches='tight')
plt.show();

In [ ]:
# '2021-12-29 00:00:00', '2022-01-05 00:00:00'
df.loc[(df['cdate']>= '2021-12-01')&(df['cdate']<'2021-12-08'), 'json'].unique()

In [ ]:
df.loc[(df['json']== '10.json'), 'cdate'].max()

In [ ]:
lst = os.listdir(data_dir)

In [ ]:
lst[40]

In [ ]:
window = 7
top_and_bottom_topics = get_top_and_bottom_topics(data_dir)
weekly_topic_distributions = collect_topic_distributions(window=window,
                                                            data_dir=data_dir, weekday_list = None, options=None)

In [ ]:
with open(f"weekly_topic_distributions.pkl","wb") as f:
    pickle.dump(weekly_topic_distributions, f)


In [ ]:
with open("weekly_topic_distributions.pkl","rb") as f:
    weekly_topic_distributions = pickle.load(f)

In [ ]:
ginis = list(map(lambda x: calculate_gini(list(x.values())), weekly_topic_distributions))

In [ ]:
entropies = list(map(lambda x: calculate_entropy(list(x.values())), weekly_topic_distributions))

In [ ]:
gini_entropy_dict = {'Gini Coefficient' : ginis, 'Entropy' : entropies}

In [ ]:
len(weekly_topic_distributions)

In [ ]:
rel_week = np.array(np.arange(-52, 52))

In [ ]:
st_0 = st.Stats(rel_week, entropies[:104], 2, 0.95)

In [ ]:
F_stat_0, p_value_0 = st_0.chow_test()

In [ ]:
print("!")

In [ ]:

### create a figure with multiple subplots
sharey = True ## 또는 sharey=False
sharex = True ## 또는 sharex=False
# g_num  = len(proportion_dict.items())

fig, axs = plt.subplots(1, 4, figsize = (24, 6), constrained_layout=True)
colors = plt.get_cmap('tab20').colors 


for x, (title, proportion) in enumerate(tot_dict.items()):
    rel_week = sorted(proportion['rel_week'].unique())
    topic_list = list(proportion['Topic'].unique())
    order_list = top_and_bottom_topics[x]
    colors = color_list[x]

    # 전체 주차 길이에 맞춰 bottom 초기화
    bottom = np.zeros(len(rel_week))

    for idx, topic in enumerate(order_list):
        # topic별 데이터
        t_p = proportion[proportion['Topic'] == topic]

        # topic별 주차별 비율을 rel_week 길이에 맞춰 채우기
        count_full = np.zeros(len(rel_week))
        for i, rw in enumerate(t_p['rel_week']):
            if rw in rel_week:
                rw_idx = rel_week.index(rw)
                count_full[rw_idx] = t_p.loc[t_p['rel_week'] == rw, 'pct'].values[0]

        # bar plot
        axs[x].bar(rel_week, count_full, bottom=bottom, label=topic, color=colors[idx], width=1.0, align='center')
        bottom += count_full  # 누적

    # 보조선 및 제목/레이블 등
    axs[x].axvline(x=0, color='tab:red', linestyle='-.', linewidth=1)
    # axs[x].set_title(f'{title}', fontsize=25)

    axs[x].text(0.5, 1.05, f"{title}",
            ha='center', va='bottom', fontsize=22, fontweight='bold', transform=axs[x].transAxes)

    axs[x].text(0.5, 1.00, "",
        ha='center', va='bottom', fontsize=15, transform=axs[x].transAxes)  

    axs[x].tick_params(axis='x', labelsize=16)
    axs[x].tick_params(axis='y', labelsize=16)

# for idx, measure in enumerate(gini_entropy_dict.keys()):
#     idx+=2
# entropy만 시각화
idx = 2
list_ = gini_entropy_dict['Entropy'][:104]
x_rel, divider = get_dist_x_div(list_)

reg_bf = calc_regression_with_ci(x_rel[:divider], list_[:divider])
reg_af = calc_regression_with_ci(x_rel[divider:], list_[divider:])

reg_bf_summary = reg_bf["pred_summary"]
reg_af_summary = reg_af["pred_summary"]

# 회귀선 (예측값)
reg_bf_y_pred = reg_bf_summary["mean"]
reg_af_y_pred = reg_af_summary["mean"]
# 신뢰구간
reg_bf_ci_lower = reg_bf_summary["mean_ci_lower"]
reg_bf_ci_upper = reg_bf_summary["mean_ci_upper"]

reg_af_ci_lower = reg_af_summary["mean_ci_lower"]
reg_af_ci_upper = reg_af_summary["mean_ci_upper"]

p_value_txt = '($p < 0.001$)' if p_value_0 <0.001 else f'($p = {p_value_0:.3f}$)'

axs[idx].scatter(x_rel, list_, color = 'darkgray', alpha = 0.7,  s=10, marker='x')
axs[idx].axvline(x=0, color='tab:red', linestyle='-.', linewidth=1)
# axs[idx].set_ylabel(f"{measure} of Topic Distribution", fontsize = 10)
axs[idx].plot(x_rel[:divider], reg_bf_y_pred, linewidth=2, label = 'before ChatGPT')
axs[idx].plot(x_rel[divider:], reg_af_y_pred, linewidth=2, label = 'after ChatGPT')

axs[idx].fill_between(x_rel[:divider], reg_bf_ci_lower, reg_bf_ci_upper, alpha=0.1)
axs[idx].fill_between(x_rel[divider:], reg_af_ci_lower, reg_af_ci_upper, alpha=0.1)

axs[idx].legend(frameon=False, loc='best', fontsize=14)
# axs[idx].set_title(f"Changes in Entropy (topic){p_value_txt}", fontsize=25)
# axs[idx].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)

axs[idx].tick_params(axis='x', labelsize=16)
axs[idx].tick_params(axis='y', labelsize=16)


axs[idx].text(0.5, 1.05, f"Changes in Entropy (topic)",
            ha='center', va='bottom', fontsize=22, fontweight='bold', transform=axs[idx].transAxes)

axs[idx].text(0.5, 1.00, f"{p_value_txt}",
        ha='center', va='bottom', fontsize=15, transform=axs[idx].transAxes)  

# for idx, measure in enumerate(gini_entropy_dict.keys()):
#     idx+=2
# entropy만 시각화
idx = 3
list_ = gini_entropy_dict['Entropy'][52:]
x_rel, divider = get_dist_x_div(list_)

reg_bf = calc_regression_with_ci(x_rel[:divider], list_[:divider])
reg_af = calc_regression_with_ci(x_rel[divider:], list_[divider:])

reg_bf_summary = reg_bf["pred_summary"]
reg_af_summary = reg_af["pred_summary"]

# 회귀선 (예측값)
reg_bf_y_pred = reg_bf_summary["mean"]
reg_af_y_pred = reg_af_summary["mean"]
# 신뢰구간
reg_bf_ci_lower = reg_bf_summary["mean_ci_lower"]
reg_bf_ci_upper = reg_bf_summary["mean_ci_upper"]

reg_af_ci_lower = reg_af_summary["mean_ci_lower"]
reg_af_ci_upper = reg_af_summary["mean_ci_upper"]

p_value_txt = '($p < 0.001$)' if p_value_0 <0.001 else f'($p = {p_value_0:.3f}$)'

axs[idx].scatter(x_rel, list_, color = 'darkgray', alpha = 0.7,  s=10, marker='x')
axs[idx].axvline(x=0, color='tab:red', linestyle='-.', linewidth=1)
# axs[idx].set_ylabel(f"{measure} of Topic Distribution", fontsize = 10)
axs[idx].plot(x_rel[:divider], reg_bf_y_pred, linewidth=2, label = 'before ChatGPT')
axs[idx].plot(x_rel[divider:], reg_af_y_pred, linewidth=2, label = 'after ChatGPT')

axs[idx].fill_between(x_rel[:divider], reg_bf_ci_lower, reg_bf_ci_upper, alpha=0.1)
axs[idx].fill_between(x_rel[divider:], reg_af_ci_lower, reg_af_ci_upper, alpha=0.1)

axs[idx].legend(frameon=False, loc='best', fontsize=14)
# axs[idx].set_title(f"Changes in Entropy (topic){p_value_txt}", fontsize=25)
# axs[idx].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)

axs[idx].tick_params(axis='x', labelsize=16)
axs[idx].tick_params(axis='y', labelsize=16)


axs[idx].text(0.5, 1.05, f"Changes in Entropy (topic)",
            ha='center', va='bottom', fontsize=22, fontweight='bold', transform=axs[idx].transAxes)

axs[idx].text(0.5, 1.00, f"{p_value_txt}",
        ha='center', va='bottom', fontsize=15, transform=axs[idx].transAxes)  

axs[0].set_ylabel("Accumulated topic share", fontsize = 22)
axs[2].set_ylabel(f"Entropy", fontsize = 22)

fig.supxlabel("Weeks relative to ChatGPT release", fontsize=22) 
plt.savefig(f"{output_dir}C_Result_Fig2_1.png", dpi=300, bbox_inches='tight')
plt.show();